# Working with pyfar Audio objects

Audio data are at the core of pyfar. Three classes can be used for storing, processing, and visualizing such data. These are:

- `pyfar.Signal` can be used to store equidistant and complete signals that can be converted between the time and frequency domain.
- `pyfar.TimeData` and `pyfar.FrequencyData` are intended for data that cannot be converted between the time and frequency domain. Examples for this are time data with missing data and frequency data that is only available at certain frequencies, e.g., third-octave data.

The following examples introduce fundamental concepts behind audio classes and show how the data inside audio objects can be accessed. See the `pyfar documentation` for a complete overview.

In [ ]:
import pyfar as pf

## pyfar Signals

*Signals* are the most versatile and frequently used audio objects and are thus covered first. In addition, many concepts of *Signals* cary over to *TimeData* and *FrequencyData*.

### Creating Signals

A *Signal* can be created in the time domain by providing the time data and sampling rate in Hz

In [ ]:
signal = pf.Signal([1, 0, 0, 0], 4)

creating a *Signal* in the frequency domain requires to explicitly specify the length of the corresponding time signal in samples and the domain

In [ ]:
signal2 = pf.Signal([1, 1, 1], 4, n_samples=4, domain='freq')

### Accessing data in Signals

The time domain data stored in audio objects can be accessed trough their `time` property

In [ ]:
signal.time

and the times in seconds at which the data was samples is contained in the `times` property

In [ ]:
signal.times

Similarly the frequency domain data is stored in the `freq` property

In [ ]:
signal.freq

and the corresponding frequencies in Hz in the `frequencies` property

In [ ]:
signal.frequencies

There are a few important things to note right away.

- data inside audio objects are stored in `numpy` arrays with at least 2-dimensions. In this example `signal` is an audio object with a single channel of audio data and the shape of `signal.time` is `(1, 4)`. In pyfar the samples of time data (4 in this example) and the frequency bins of frequency data are always stored in the last dimension of `signal.time` and `signal.freq`
- The conversion between the time and frequency domain automatically happens upon request by the underlying *Signal* class. Internally the data is always stored in the domain that was last used and converted using the Discrete Fourier Transform.

It is also **important to note** that the data returned by the `freq` property can depend on the normalization of the Discrete Fourier Transform as explained in more detail `here`. The frequency data without any normalization can be accessed using the `freq_raw` property. However, for this example the two return the same results

In [ ]:
signal.freq_raw

Read on to learn how to find out which normalization of the Discrete Fourier Transform is used for a signal.

The indices of a certain frequency or frequency range an be found using

In [ ]:
index = signal.find_nearest_frequency(1)
print(index)

and in analogy `signal.find_nearest_time` can be used to find a certain point in time.

### Write data to Signals

The same properties can be used to set the data inside audio objects.

In [ ]:
signal.time = [2, 0, 0, 0]
print(f'signal.time = {signal.time}')


In [ ]:
signal2.freq = [2, 2, 2]
print(f'signal2.freq = {signal2.freq}')

### Additional information stored in Signals

Converting between time and frequency data is already useful, but audio objects can do more. They contain additional information about their data that can be helpful. Some basic information is already returned when prompting an audio object

In [ ]:
signal

Lets break down the information given by the *Signal* one by one. 

**Domain:** The signal is in the time domain, because we last accessed its time data above. We can get and set this property with the `signal.domain` attribute but it is of not much interest when working with Signal objects because they are automatically converted between the time and frequency domain, when it is required.

In [ ]:
signal.domain

**Signal Type:** It is an *energy* Signal, which is important to note and understand. Energy signals are finite signals with finite energy. Examples for these kinds of signals are impulse responses. *Power* signals on the other hand have an infinite duration and energy. Examples are noise or sine signals of which we typically observe a block of finite size. The signal type is a read only property that is set by the DFT normalization introduced below.

In [ ]:
signal.signal_type

**DFT Normalization:** For *power* Signals, different DFT normalization can be used that differently scale the values stored in `signal.freq`. For *energy* signals, the DFT normalization is always 'none'. For more information refer to the `Fast Fourier Transform` examples.

In [ ]:
signal.fft_norm

### Signal cshape, length, and caxis

Signals can contain multiple channels of audio data and the shape of the data inside the audio object is often important. There are multiple attributes describing this.

**Channel shape:** The *channel shape*, in short `cshape` gives the shape of the data inside a *Signal* but ignores the number of samples or frequency bins. For example our first signal created above has one channel and 4 samples, hence the data is of shape `(1, 4)`. Because the `cshape` ignores the samples it is `(1, )`. There are two ideas behind this. First, digital signal processing methods often work on channels, and second the length of the signal might be different in the time and frequency domain.

In [ ]:
signal.cshape

**Length:** The signal length in samples and bins is stored in the properties

In [ ]:
signal.n_samples

and

In [ ]:
signal.n_bins

Note that only the half sided spectrum is stored for real valued time signals. Hence the length differs in the time and frequency domain in this case.

**Channel axis:** Some functions in pyfar operate along one or more axes of the data. In analogy, to the channel shape, these axes are referred to as *channel axis* or in short `caxis`. Consider a signal of `cshape=(3, 4)` and a duration of `n_samples=128`. In this case `caxis=0` refers to the first dimension of size 3 and `caxis=-1` refers to the last dimension of size 4 but *not* the dimension containing the 128 samples.

### Multi-Channel Signals

*Signals* may contain data in multiple channels and pyfar offers some functionality to ease handling such signals. Let the following signal of `cshape=(2, 3)` symbolize data obtained for 2 loudspeaker and 3 microphone positions

In [ ]:
signal3 = pf.Signal(
    [[[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]],
     [[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]]],
    6)

signal3

A subset of the channels can be accessed through slicing. For example the first channel in both dimensions can be obtained by

In [ ]:
channel = signal3[0, 0]
channel

the output shows that the sloce is *Signal* as well, which is often useful because most pyfar functions require *Signals* as input. This also works for obtaining larger slices, for example

In [ ]:
channels = signal3[0]
channels

returns the data for the first loudspeaker and all three microphone positions.

## pyfar TimeData and FrequencyData

To create *TimeData*, specify the data and the times at which the data was sampled

In [ ]:
# time data with non-equidistant sampling times of 0, 1, and 3 seconds
time = pf.TimeData([1, 0, -1], [0, 1, 3])

and creating *FrequencyData* requires the specification of the frequencies that belong to the data

In [ ]:
# frequency data containing values at 400, 800, and 1600 Hz
frequency = pf.FrequencyData([1, .8, .7], [400, 800, 1600])

This showed that creating *TimeData* and *FrequencyData* is different from creating *Signals*. The differences between the audio objects are also reflected in their print:

In [ ]:
time


In [ ]:
frequency


shows that *TimeData* and *FrequencyData* do not have a sampling rate and FFT normalization. Apart from this the different audio objects behave very similar. In fact the `pyfar.Signal` class is derived from `pyfar.TimeData` and `pyfar.FrequencyData`. Hence, the `time` property can be used to access *TimeData*

In [ ]:
time.time

 and the `freq` property can be used to access *FrequencyData*

In [ ]:
frequency.freq

In general, all time domain functionality of *Signals* that was introduced above is available for *TimeData* and all frequency domain functionality is available for *FrequencyData*.

## Further reading

After getting used to adding and accessing data in audio objects, you are ready for discovering ways of inspecting and working with audio objects. Good next steps might be learning how to graphically display (plot) data from audio objects as detailed `here` and check out how to apply simple arithmetic operations with audio objects as described `here`.